In [ ]:
# 승자독식 모노폴리
# https://www.codetree.ai/training-field/frequent-problems/problems/odd-monopoly/description?page=2&pageSize=20




# personList : 각 사람들 정보 관리 리스트 - 각 사람마다 [pose, 방향, 생존여부]
# poseMat X : 어차피 독점 여부만 확인함 (다른 사람의 위치는 참고하지 않음)
# personDir (고정): 각 사람 우선 순위 방향 리스트 (사람 인덱스 - 방향)

# monoMat : 독점 여부 mat - 사람 숫자로 표시 (초기화는 -1)
# timeMat : 시간 mat


n, m, k = list(map(int, input().split()))

poseMat = [[0 for i in range(n)] for j in range(n)]
personList = []
for r in range(n):
    poseMat[r] = list(map(int, input().split()))
    for c in range(n):
        if (poseMat[r][c] == 0):
            poseMat[r][c] = -1
        elif (poseMat[r][c] > 0):
            poseMat[r][c] -= 1
            personList.append([poseMat[r][c], [r, c], True])  # [index, pose, 우선순위 방향, 생존여부]

personList.sort(key=lambda x: x[0])
dirList = list(map(int, input().split()))
for i in range(len(dirList)):
    personList[i].insert(2, dirList[i] - 1)

personDir = []
perPerson = []
for i in range(4 * m):
    dirList = list(map(int, input().split()))
    perPerson.append([x-1 for x in dirList])

    if (i % 4 == 3):
        personDir.append(perPerson)
        perPerson = []


monoMat = [[-1 for i in range(n)] for j in range(n)]
timeMat = [[0 for i in range(n)] for j in range(n)]
for eachPerson in personList:
    idx = eachPerson[0]
    r, c = eachPerson[1]
    monoMat[r][c] = idx
    timeMat[r][c] = k

dys = [-1, 1, 0, 0]
dxs = [0, 0, -1, 1]



def in_range(y,x):
    return 0<=y and y<n and 0<=x and x<n

# start = False
turn = 0
while turn < 1000:
    turn += 1

    # 임시 mat 각 칸에 리스트로 오는 사람들 저장
    temMat = [[[] for i in range(n)] for j in range(n)]

    for idx in range(len(personList)):
        # 각 사람마다
        # 방향 정하기
        # 현재 바라보는 방향일 경우의 우선순위 리스트 가져오기
        # 각 방향 순서대로 검사해서 본인에게 인접한 상하좌우 4 칸 중 아무도 독점계약을 맺지 않은 칸으로 이동
        # 그러한 칸이 없을 경우에는 인접한 4방향 중 본인이 독점계약한 땅으로 이동

        personIdx, pose, direction, alive = personList[idx]

        if (alive == False):
            continue

        cy, cx = pose
        # print(personIdx, direction)
        temDirList = personDir[personIdx][direction]
        # cur_dy, cur_dx = [], []
        # for each in temDirList: # 해당 방향을 바라보고 있을 때의 우선순위 정리
        #     cur_dy.append(dys[each])
        #     cur_dx.append(dxs[each])

        dir_decide = -1
        ny_decide,nx_decide = -1, -1

        # print(personIdx)
        # print(cur_dy, cur_dx)

        for dir_idx in temDirList:
            dy = dys[dir_idx]
            dx = dxs[dir_idx]

            ny, nx = cy + dy, cx + dx
            # print(cy, cx, dy, dx)

            if(not in_range(ny, nx)):
                continue

            if (monoMat[ny][nx] == -1):
                dir_decide = dir_idx
                ny_decide,nx_decide = ny, nx
                break

        if (dir_decide == -1):
            for dir_idx in temDirList:
                dy = dys[dir_idx]
                dx = dxs[dir_idx]
                ny, nx = cy + dy, cx + dx

                if (not in_range(ny, nx)):
                    continue

                if (monoMat[ny][nx] == personIdx):
                    dir_decide = dir_idx
                    ny_decide, nx_decide = ny, nx
                    break

        if (dir_decide == -1): # 갈 곳 없으면 continue - 이게 있을수있나
            print('못감')
            continue

        # 방향 업데이트
        personList[idx][2] = dir_decide

        # temMat에 갈곳 임시 저장
        temMat[ny_decide][nx_decide].append(personIdx)


    # mat 각 칸에 len이 1보다 크면 가장 작은 번호를 가진 플레이어만 살아남기, 나머지는 생존 False
    for r in range(n):
        for c in range(n):

            if(len(temMat[r][c]) == 0):
                continue
            # print(temMat[r][c])

            temMat[r][c].sort()
            while(len(temMat[r][c]) > 1):
                disapperIdx = temMat[r][c].pop()
                personList[disapperIdx][-1] = False # 생존 여부 False

            # 이동한 사람 pose 정보 업데이트
            aliveIdx = temMat[r][c][0]
            personList[aliveIdx][1] = [r,c]




    # timeMat 0보다 큰 애들 1씩 감소
    for r in range(n):
        for c in range(n):
            if timeMat[r][c] > 0:
                timeMat[r][c] -= 1

                if(timeMat[r][c] == 0): # 1 감소해서 0이면 monoMat에서 -1로 표시
                    monoMat[r][c] = -1

    ### monoMat, timeMat
    for idx in range(len(personList)):
        personIdx, pose, direction, alive = personList[idx]
        y, x = pose

        if(alive == False):
            continue

        # monoMat에 새로 사람 idx값 추가
        monoMat[y][x] = personIdx

        # timeMat에 새로운 값 업데이트
        timeMat[y][x] = k

    #
    # print()
    # print(turn, '====================================')
    #
    # for i in personList:
    #     print(i)
    # print()
    #
    # # for i in range(len(personDir)):
    # #     print(personDir[i])
    # # print()
    #
    # for i in range(n):
    #     print(monoMat[i])
    # print()
    #
    # for i in range(n):
    #     print(timeMat[i])
    # print()



    # 만약 남은 사람이 1번만 있으면 끝
    finish = True
    for idx in range(len(personList)):
        if(personList[idx][-1] == True and idx != 0):
            finish = False
            break
    if(finish == True):
        break

    # break

if(turn >= 1000):
    print(-1)
else:
    print(turn)